In [1]:
# Verify if there is a match with the original XML files
import os
path_data = "C:/Users/USER/eRisk2017/2017/"
# Get the list of the first chunk and check if it is present in the rest
individuals_train_p = list(os.listdir(path_data+"/train/positive_examples_anonymous_chunks/chunk 1"))
individuals_train_p = [individual.replace('1.xml','') for individual in individuals_train_p if individual !='desktop.ini']
print(len(individuals_train_p))
print('\nPositive training responses')
for chunk in range(1, 11):
    individuals_in_chunk_p = os.listdir(path_data+'/train/positive_examples_anonymous_chunks/chunk '+str(chunk))
    individuals_in_chunk_p = [individual.replace(str(chunk)+'.xml','') for individual in individuals_in_chunk_p if individual !='desktop.ini']
    print('chunk:', chunk, '\t', set(individuals_train_p) == set(individuals_in_chunk_p))
individuals_train_n = list(os.listdir(path_data+"/train/negative_examples_anonymous_chunks/chunk 1"))
individuals_train_n = [individual.replace('1.xml','') for individual in individuals_train_n if individual !='desktop.ini']
print(len(individuals_train_n))
print('\nNegative training responses')
for chunk in range(1, 11):
    individuals_in_chunk_n = os.listdir(path_data+'/train/negative_examples_anonymous_chunks/chunk '+str(chunk))
    individuals_in_chunk_n = [individual.replace(str(chunk)+'.xml','') for individual in individuals_in_chunk_n if individual !='desktop.ini']
    print('chunk:', chunk, '\t', set(individuals_train_n) == set(individuals_in_chunk_n))
individuals_train_t = list(os.listdir(path_data+'/test/chunk 1/chunk 1/'))
individuals_train_t = [individual.replace('1.xml','') for individual in individuals_train_t if individual !='desktop.ini']
print(len(individuals_train_t))
print('\nTest training responses')
for chunk in range(1, 11):
    individuals_in_chunk_t = os.listdir(path_data+'/test/chunk '+str(chunk)+'/chunk '+str(chunk))
    individuals_in_chunk_t = [individual.replace(str(chunk)+'.xml','') for individual in individuals_in_chunk_t if individual !='desktop.ini']
    print('chunk:', chunk, '\t', set(individuals_train_t) == set(individuals_in_chunk_t))

# Corpus Processing
print("Corpus Processing")
print("Corpus Features:")
print("Reddit user posts")
print("Separated into 10 chunks")
print("XML Format")
print("Training Data")

83

Positive training responses
chunk: 1 	 True
chunk: 2 	 True
chunk: 3 	 True
chunk: 4 	 True
chunk: 5 	 True
chunk: 6 	 True
chunk: 7 	 True
chunk: 8 	 True
chunk: 9 	 True
chunk: 10 	 True
403

Negative training responses
chunk: 1 	 True
chunk: 2 	 True
chunk: 3 	 True
chunk: 4 	 True
chunk: 5 	 True
chunk: 6 	 True
chunk: 7 	 True
chunk: 8 	 True
chunk: 9 	 True
chunk: 10 	 True
401

Test training responses
chunk: 1 	 True
chunk: 2 	 True
chunk: 3 	 True
chunk: 4 	 True
chunk: 5 	 True
chunk: 6 	 True
chunk: 7 	 True
chunk: 8 	 True
chunk: 9 	 True
chunk: 10 	 True
Corpus Processing
Corpus Features:
Reddit user posts
Separated into 10 chunks
XML Format
Training Data


In [2]:
# the folder had different no. of files. to debug :
# individuals_train_n = list(os.listdir(path_data+"/train/negative_examples_anonymous_chunks/chunk 1"))
# individuals_train_n = [individual.replace('1.xml','') for individual in individuals_train_n if individual !='desktop.ini']
# print(len(individuals_train_n))
# print('\nNegative training responses')
# for chunk in range(1, 11):
#     individuals_in_chunk_n = os.listdir(path_data+'/train/negative_examples_anonymous_chunks/chunk '+str(chunk))
#     individuals_in_chunk_n = [individual.replace(str(chunk)+'.xml','') for individual in individuals_in_chunk_n if individual !='desktop.ini']
#     print('chunk:', chunk, '\t', len(set(individuals_train_n)), len(set(individuals_in_chunk_n)))
# #     print([i for i in set(individuals_train_n) if i not in set(individuals_in_chunk_n)])
#     print([i for i in set(individuals_in_chunk_n) if i not in set(individuals_train_n)])
# #     print(set(individuals_train_n))
# #     print(set(individuals_in_chunk_n))

In [3]:
import re
import os
mypath = "C:/Users/USER/eRisk2017/2017/"
os.chdir(mypath)
from os import listdir
from glob import glob  # List files

# Scrapping
from bs4 import BeautifulSoup as Soup
import requests
import pandas as pd
import numpy as np

In [4]:
# Preprocessing
import nltk
# Convert to words
from nltk.tokenize import word_tokenize
# List of stopwords
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
# Stemming
from nltk.stem import PorterStemmer
ps = PorterStemmer()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [5]:
def corpus_process_train(nChunks, stemm=True, stopword=True, maxlenword=15):
    
    lengths=[]
    # Root Directories
    root_p = mypath +'train/positive_examples_anonymous_chunks/chunk '
    root_n = mypath +'train/negative_examples_anonymous_chunks/chunk '
    
    # List of subjects by response type
    subjects_p = [subject.replace('_1.xml', '') for subject in listdir(root_p + '1/') if subject != 'desktop.ini']
    subjects_n = [subject.replace('_1.xml', '') for subject in listdir(root_n + '1/') if subject != 'desktop.ini']
    
    # Container DataFrame
    # Initially created with only subject IDs and their labels
    df_train = pd.DataFrame(subjects_p + subjects_n, columns=['Id'])  # Subject IDs
    df_train['Depress'] = len(subjects_p) * [1] + len(subjects_n) * [0]  # Depression label
    
    # Process each chunk to generate global data
    
    for chunk in range(1, nChunks + 1):
        subjects_p_path = [root_p + str(chunk) + '/' + subject + '_' + str(chunk) + '.xml' for subject in subjects_p]
        subjects_n_path = [root_n + str(chunk) + '/' + subject + '_' + str(chunk) + '.xml' for subject in subjects_n]
        subjects_train_path = subjects_p_path + subjects_n_path
        chunk_posts = []
        
        for subject in subjects_train_path:
            posts_subject = []
            infile = open(subject, 'r', encoding='utf8')
            post_raw = Soup(infile.read())
            posts = post_raw.find_all('text')  # Get all posts
            
            for post in posts:
                if post.getText() != None:
                    posts_subject.append(post.getText())
                    lengths.append(len(posts_subject))
######################################### Preprocessing ############################################################
            # 1. Lowercase  2. Stemming 3. Words only        
            all_post_subject = ' '.join(posts_subject)
            # all_post_subject = re.sub(r'http\S+', '', all_post_subject)  # Remove web addresses, is.alpha() performs the task
            tokens_all_post_subject = word_tokenize(all_post_subject)    # Tokenize
            tokens_subject = [token.lower() for token in tokens_all_post_subject]
            
            if stemm:
                if stopword:
                    tokens_subject = [ps.stem(token) for token in tokens_subject if token.isalpha() and not token in stop_words and len(token) < maxlenword]
                else:
                    tokens_subject = [ps.stem(token) for token in tokens_subject if token.isalpha() and len(token) < maxlenword]
            else:
                if stopword:
                    tokens_subject = [token for token in tokens_subject if token.isalpha() and not token in stop_words and len(token) < maxlenword]
                else:
                    tokens_subject = [token for token in tokens_subject if token.isalpha() and len(token) < maxlenword]
            
            post_clean = ' '.join(tokens_subject)  # Join all the posts
            chunk_posts.append(post_clean)
        
        name_chunk = 'Chunk_' + str(chunk)
        df_train[name_chunk] = chunk_posts
   
    return df_train, lengths

In [6]:
def corpus_process_test(nChunks, stemm=True, stopword=True, maxlenword=15):
    lengths=[]
    # Process labels
    dir_data_test = "C:/Users/USER/eRisk2017/2017/test/test_golden_truth.txt"
    y_test = pd.read_csv(dir_data_test, header=None, sep='\t')
#     print (y_test)
#     y_test = y_test.drop([0], axis=1)
#     print(y_test)
    y_test.columns = ['Id', 'Depress']

    # Root directory
    root_p = mypath+'test/chunk '
    # List of subjects by response type
    subjects_t = [subject.replace('_1.xml', '') for subject in listdir(root_p+'1/chunk 1/') if subject != 'desktop.ini']
#     print(subjects_t)
#     subjects_t=subjects_t[0:2]
#     print(subjects_t)

    # Container DataFrame
    df_train = pd.DataFrame(subjects_t, columns=['Id'])  # Subject IDs

    # Process each chunk to generate global data
    for chunk in range(1, nChunks+1):
        subjects_t_path = [root_p + str(chunk) +'/chunk '+ str(chunk) + '/' + subject + '_' + str(chunk) + '.xml' for subject in subjects_t]
        chunk_posts = []
        for subject in subjects_t_path:
            posts_subject = []
#             print(subject)
            infile = open(subject, 'r', encoding='utf8')
            post_raw = Soup(infile.read())
#             print(post_raw)
            posts = post_raw.find_all('writing')  # Get all posts
            for post in posts:
                if post.find('title').getText() is not None:
                    posts_subject.append(post.find('title').getText())
                if post.find('text').getText() is not None:
                    posts_subject.append(post.find('text').getText())
                lengths.append(len(posts_subject))
#             print(posts_subject)
            #########################################Preprocessing############################################################
            # 1. Lowercase  2. Stemming 3. Words only
            all_post_subject = ' '.join(posts_subject)
#             print(all_post_subject)
            # all_post_subject=re.sub(r'http\S+', '', all_post_subject)  # Remove web addresses, .isalpha() does the task
            tokens_all_post_subject = word_tokenize(all_post_subject)  # Tokenize
            tokens_subject = [token.lower() for token in tokens_all_post_subject]
#             print(tokens_subject)
            if stemm:
                if stopword:
                    tokens_subject = [ps.stem(token) for token in tokens_subject if
                                      token.isalpha() and not token in stop_words and len(token) < maxlenword]
                else:
                    tokens_subject = [ps.stem(token) for token in tokens_subject if
                                      token.isalpha() and len(token) < maxlenword]
            else:
                if stopword:
                    tokens_subject = [token for token in tokens_subject if
                                      token.isalpha() and not token in stop_words and len(token) < maxlenword]
#                     print(tokens_subject)
                else:
                    tokens_subject = [token for token in tokens_subject if token.isalpha() and len(token) < maxlenword]
            post_clean = ' '.join(tokens_subject)  # Join all posts
#             print(post_clean)
            chunk_posts.append(post_clean)
        name_chunk = 'Chunk_' + str(chunk)
#         print(chunk_posts)
        df_train[name_chunk] = chunk_posts
#     print(y_test['Id'].dtypes)
#     print(df_train['Id'].dtypes)
    y_test['Id'] = y_test['Id'].str.strip()
    df_train['Id'] = df_train['Id'].str.strip()
    df_test = pd.merge(y_test, df_train, on='Id', how='inner')
    return df_test,lengths


In [7]:
df_train,lengths=corpus_process_train(10,stemm=False)

In [8]:
np.mean(lengths),np.max(lengths), np.min(lengths), np.median(lengths)

(63.18914796473495, 208, 1, 54.0)

In [9]:
df_test, lengths =corpus_process_test(10,stemm=False)
np.mean(lengths),np.max(lengths), np.min(lengths), np.median(lengths)

(118.88654699603589, 416, 2, 100.0)

In [27]:
import matplotlib.pyplot as plt
lengths=[]
for i in range(1,11):
    ttrain = df_train['Chunk_'+str(i)]
    lengths.append(ttrain.str.len())

In [28]:
np.mean(lengths),np.max(lengths), np.min(lengths), np.median(lengths)

(4990.519547325103, 82324, 0, 1784.0)

In [29]:
import matplotlib.pyplot as plt
lengths=[]
for i in range(1,11):
    ttrain = df_test['Chunk_'+str(i)]
    lengths.append(ttrain.str.len())

In [30]:
np.mean(lengths),np.max(lengths), np.min(lengths), np.median(lengths)

(6267.272319201995, 168410, 0, 3194.5)

In [9]:
df_train [0:2]

,Id,Depress,Chunk_1,Chunk_2,Chunk_3,Chunk_4,Chunk_5,Chunk_6,Chunk_7,Chunk_8,Chunk_9,Chunk_10
0,train_subject1095,1,last line pretty much first could course wasnt...,dammit exact day lykke li kansas city already ...,kinda wish like vertigo lucifer would pulled m...,last night finally decided get slime king summ...,could sworn read somewhere montana highway spe...,get campus shooting guys google play billed ti...,know cinemark chain cheap refills marathon tic...,convergence convergence nightwing oracle conve...,well lightning bolt left bought classic pack g...,believe get next week dark horse neverboy dc c...
1,train_subject1190,1,sure get flamed true friends write life choice...,interesting would nice notified people complai...,live close libbie mill really started developi...,way right use zoom lens series primes instead ...,hey live richmond area itching road trip think...,used work police department definitely lightin...,yep hoping kids decent human beings help paren...,point posting seems like kind dick thing hey l...,neighbor sitting backyard last night farting r...,passing offer call ask gold gym posted offer f...


In [10]:
df_test [0:2]

,Id,Depress,Chunk_1,Chunk_2,Chunk_3,Chunk_4,Chunk_5,Chunk_6,Chunk_7,Chunk_8,Chunk_9,Chunk_10
0,test_subject9942,1,know satanic music system based rules works ge...,used problem lane presence get past cc comes p...,jump people insist form counter play lol say e...,transtrenders kind specifically saying nonbina...,gender exists social construct even observable...,messed okay hmm okay helpful thank insight fac...,withdraw class know cases people sure failing ...,pan people want identify bi special snowflake ...,think feminism hurts asexuals namely male asex...,right knowing truth feminism suddenly guys one...
1,test_subject3986,1,thanks reminding forgot put nyx eyeliner produ...,wow color looks good recommendation make sure ...,haha highschool senior picture yearbook eyebro...,ah another fellow raccoon eyed eighth grader l...,cosmetics celebration foundation cruelty free ...,valentine day look love leaves scars love blin...,actually returned back japan two months ago li...,years older younger sister born could never re...,trying pursue teaching abroad asia programs re...,guess living small town nothing makes everythi...


In [11]:
df_test.isnull().sum()

Id          0
Depress     0
Chunk_1     0
Chunk_2     0
Chunk_3     0
Chunk_4     0
Chunk_5     0
Chunk_6     0
Chunk_7     0
Chunk_8     0
Chunk_9     0
Chunk_10    0
dtype: int64

In [12]:
# save data
df_train.to_csv('train_Depression_all_chunks_nosteem.csv', encoding='utf-8', na_rep=' ')
df_test.to_csv('test_Depression_all_chunks_nosteem.csv')

In [13]:
train_df=pd.read_csv('train_Depression_all_chunks_nosteem.csv')
train_df=train_df.replace(np.nan, '', regex=True)
test_df=pd.read_csv('test_Depression_all_chunks_nosteem.csv')
test_df=train_df.replace(np.nan, '', regex=True)

In [14]:
train_df.isnull().sum()

Unnamed: 0    0
Id            0
Depress       0
Chunk_1       0
Chunk_2       0
Chunk_3       0
Chunk_4       0
Chunk_5       0
Chunk_6       0
Chunk_7       0
Chunk_8       0
Chunk_9       0
Chunk_10      0
dtype: int64

In [15]:
train_df.isnull().sum()

Unnamed: 0    0
Id            0
Depress       0
Chunk_1       0
Chunk_2       0
Chunk_3       0
Chunk_4       0
Chunk_5       0
Chunk_6       0
Chunk_7       0
Chunk_8       0
Chunk_9       0
Chunk_10      0
dtype: int64